In [56]:
import numpy as np
import pandas as pd
import json

In [57]:
lg = pd.read_json('data_legitimate_36400.json')
lg.head(10)

,0
0,http://www.conceptdraw.com/How-To-Guide/Local-...
1,https://www.edrawsoft.com/Local-Area-Network.php
2,http://www.webopedia.com/TERM/L/local_area_net...
3,https://www.acsac.org/secshelf/book001/16.pdf
4,http://www.diffen.com/difference/LAN_vs_WAN
5,http://sentence.yourdictionary.com/lan
6,http://www.itrelease.com/2012/12/examples-and-...
7,http://www.dictionary.com/browse/lan
8,http://fcit.usf.edu/Network/chap1/chap1.htm
9,http://www.Cisco.com/c/en/us/support/wireless/...


In [58]:
ph = pd.read_json('data_phishing_37175.json')
ph.head(10)

,0
0,apple-iforget.com
1,safe-id-login.com
2,manage.netflix.com.usermanagement.key.1973573....
3,manage.netflix.com.usermanagement.key.1973574....
4,helpplusinfo01.com
5,icloudsupporto.com
6,fbverify.biz
7,paypal-my-cash.com
8,scotiabank-security.com
9,user-appleid-find.com


### Changing column name

In [59]:
lg.rename(columns = {0 : 'URLs'}, inplace = True)
print (len(lg))
lg.head(10)

36400


,URLs
0,http://www.conceptdraw.com/How-To-Guide/Local-...
1,https://www.edrawsoft.com/Local-Area-Network.php
2,http://www.webopedia.com/TERM/L/local_area_net...
3,https://www.acsac.org/secshelf/book001/16.pdf
4,http://www.diffen.com/difference/LAN_vs_WAN
5,http://sentence.yourdictionary.com/lan
6,http://www.itrelease.com/2012/12/examples-and-...
7,http://www.dictionary.com/browse/lan
8,http://fcit.usf.edu/Network/chap1/chap1.htm
9,http://www.Cisco.com/c/en/us/support/wireless/...


In [60]:
ph.rename(columns = {0 : 'URLs'}, inplace = True)
ph.head(10)

,URLs
0,apple-iforget.com
1,safe-id-login.com
2,manage.netflix.com.usermanagement.key.1973573....
3,manage.netflix.com.usermanagement.key.1973574....
4,helpplusinfo01.com
5,icloudsupporto.com
6,fbverify.biz
7,paypal-my-cash.com
8,scotiabank-security.com
9,user-appleid-find.com


### Adding a column 'isPhishing' to store the Phishing Result

#### isPhishing = 0 for legitimate websites
#### isPhishing = 1 for phishing websites


In [61]:
lg["isPhishing"] = 0
lg.sort_values('URLs', inplace = True)
lg.drop_duplicates(inplace = True, subset = 'URLs')
lg.head(10)


,URLs,isPhishing
21891,http://0cu203.bankruptcy-law-help.com/15593752...,0
21795,http://1-drivers.org/videocards/diamond_multim...,0
5135,http://1000islandswritersfestival.ca/wp-conten...,0
603,http://1000projects.org/fundamentals-of-passiv...,0
7998,http://1000projects.org/technical-seminar-topi...,0
10734,http://103.ip-144-217-81.net/documents/graphic...,0
36378,http://103.ip-144-217-81.net/documents/search-...,0
27969,http://103.ip-144-217-81.net/documents/the-hel...,0
17670,http://10beasts.com/best-3d-pens/,0
11321,http://10beasts.com/best-wireless-routers/,0


In [62]:
ph["isPhishing"] = 1
ph.sort_values('URLs', inplace = True)

ph.drop_duplicates(inplace = True, subset = 'URLs')
ph.head(10)

,URLs,isPhishing
111,13bmosecure.com,1
155,34zy2.sljtm.com,1
108,accountverification.online,1
50,apaypal.co/Secure/PP/Paypal/,1
33,app-findmyiphone.com,1
142,apple-accounts.me/clients/home/,1
39,apple-icloudcid.com,1
54,apple-id-support.info,1
20359,apple-iforget.com,1
166,apple-inc-help.com,1


### Merging the 2 dataframes into a single dataframe for pre-processing

In [63]:
df = pd.concat([ph, lg])


print (len(df))
df.head(10)

54763


,URLs,isPhishing
111,13bmosecure.com,1
155,34zy2.sljtm.com,1
108,accountverification.online,1
50,apaypal.co/Secure/PP/Paypal/,1
33,app-findmyiphone.com,1
142,apple-accounts.me/clients/home/,1
39,apple-icloudcid.com,1
54,apple-id-support.info,1
20359,apple-iforget.com,1
166,apple-inc-help.com,1


## Feature Selection

### Calculating Raw Length of the urls

In [64]:
import re, statistics
def rawlength(s):
    return len(s)

def isHTTPS(s):
    if(s.startswith('https')):
        return 1
    else:
        return 0

def hasWWW(s):
    if 'www' in s:
        return 1
    else:
        return 0

def hasCOM(s):
    if '.com' in s:
        return 1
    else:
        return 0

def lenMaxWord(s):
    words = re.split('[. -/_]',s)
    m = 0
    for i in words:
        m = max(m, len(i))
    return m


def lenAvgWord(s):
    words = re.split('[. -/_]',s)
    l = []
    for i in words:
        if (len(i)>0):
            l.append(len(i))
    return round(sum(l)/len(l))

def lenMinWord(s):
    words = re.split('[. -/_]',s)
    l = []
    for i in words:
        if (len(i)>0):
            l.append(len(i))
    return min(l)

def SDlengthWord(s):
    words = re.split('[. -/_]',s)
    l = []
    for i in words:
        if (len(i)>0):
            l.append(len(i))
    return statistics.stdev(l)

def numOfSubdom(site):
    s = re.search("//",site)
    if s:
        site = site[s.end():]
    e = re.search("/",site)
    if e:
        site = site[:e.end()-1]
    site = site.split(".")
    # one for domain extension eg. com,edu,in 
    num = len(site)-1
    if 'www' in site:
        num = num-1
    return num

def keyCount(site):
    count = 0
    wordList = re.split(r"\.|\/|-|\?|=|#|&|_|@|:\/\/",site)
    for word in wordList:
        if word in keywordList:
            count += 1
    return count

df["RawLength"] = df["URLs"].apply(rawlength)
df["HTTPS"] = df["URLs"].apply(isHTTPS)
df["WWW"] = df["URLs"].apply(hasWWW)
df[".COM"] = df["URLs"].apply(hasCOM)
df["largestWord"] = df["URLs"].apply(lenMaxWord)
df["avgWord"] = df["URLs"].apply(lenAvgWord)
df["smallestWord"] = df["URLs"].apply(lenMinWord)
df["SDlength"] = df["URLs"].apply(SDlengthWord)
df["countSubDomain"] = df["URLs"].apply(numOfSubdom)

df.head(10)

,URLs,isPhishing,RawLength,HTTPS,WWW,.COM,largestWord,avgWord,smallestWord,SDlength,countSubDomain
111,13bmosecure.com,1,15,0,0,1,11,7,3,5.656854,1
155,34zy2.sljtm.com,1,15,0,0,1,5,4,3,1.154701,2
108,accountverification.online,1,26,0,0,0,19,12,6,9.192388,1
50,apaypal.co/Secure/PP/Paypal/,1,28,0,0,0,7,5,2,2.408319,1
33,app-findmyiphone.com,1,20,0,0,1,12,6,3,5.196152,1
142,apple-accounts.me/clients/home/,1,31,0,0,0,8,5,2,2.387467,1
39,apple-icloudcid.com,1,19,0,0,1,9,6,3,3.055050,1
54,apple-id-support.info,1,21,0,0,0,7,4,2,2.081666,1
20359,apple-iforget.com,1,17,0,0,1,7,5,3,2.000000,1
166,apple-inc-help.com,1,18,0,0,1,5,4,3,0.957427,1


#### Loading keywords

In [65]:
key = open("keywords.txt", "r")
keys = key.read()
keywords = keys.split('\n')

#### Load Brand Name

In [66]:
brand = open("BrandName.txt", "r")
brandname = brand.read()
brands = brandname.split('\n')

#### Check if brandname in URL

In [67]:
def checkBrandName(s):
    for i in brands:
        if i.lower() in s.lower():
            return 1
    return 0
df["brandName"] = df["URLs"].apply(checkBrandName)

In [68]:
df.head(10)

,URLs,isPhishing,RawLength,HTTPS,WWW,.COM,largestWord,avgWord,smallestWord,SDlength,countSubDomain,brandName
111,13bmosecure.com,1,15,0,0,1,11,7,3,5.656854,1,0
155,34zy2.sljtm.com,1,15,0,0,1,5,4,3,1.154701,2,0
108,accountverification.online,1,26,0,0,0,19,12,6,9.192388,1,0
50,apaypal.co/Secure/PP/Paypal/,1,28,0,0,0,7,5,2,2.408319,1,1
33,app-findmyiphone.com,1,20,0,0,1,12,6,3,5.196152,1,1
142,apple-accounts.me/clients/home/,1,31,0,0,0,8,5,2,2.387467,1,1
39,apple-icloudcid.com,1,19,0,0,1,9,6,3,3.055050,1,1
54,apple-id-support.info,1,21,0,0,0,7,4,2,2.081666,1,1
20359,apple-iforget.com,1,17,0,0,1,7,5,3,2.000000,1,1
166,apple-inc-help.com,1,18,0,0,1,5,4,3,0.957427,1,1


#### Count number of times a brand name appears

In [69]:
def countBrandName(s):
    count = 0
    for i in brands:
        if i.lower() in s.lower():
            count +=1
            
    return count

df["countBrand"] = df["URLs"].apply(countBrandName)


In [70]:
df.head(10)

,URLs,isPhishing,RawLength,HTTPS,WWW,.COM,largestWord,avgWord,smallestWord,SDlength,countSubDomain,brandName,countBrand
111,13bmosecure.com,1,15,0,0,1,11,7,3,5.656854,1,0,0
155,34zy2.sljtm.com,1,15,0,0,1,5,4,3,1.154701,2,0,0
108,accountverification.online,1,26,0,0,0,19,12,6,9.192388,1,0,0
50,apaypal.co/Secure/PP/Paypal/,1,28,0,0,0,7,5,2,2.408319,1,1,1
33,app-findmyiphone.com,1,20,0,0,1,12,6,3,5.196152,1,1,1
142,apple-accounts.me/clients/home/,1,31,0,0,0,8,5,2,2.387467,1,1,1
39,apple-icloudcid.com,1,19,0,0,1,9,6,3,3.055050,1,1,2
54,apple-id-support.info,1,21,0,0,0,7,4,2,2.081666,1,1,1
20359,apple-iforget.com,1,17,0,0,1,7,5,3,2.000000,1,1,1
166,apple-inc-help.com,1,18,0,0,1,5,4,3,0.957427,1,1,1


#### Count of keywords

In [71]:
def countKeyword(s):
    count = 0
    for i in keywords:
        if i.lower() in s.lower():
            count +=1
            
    return count

df["keywordCount"] = df["URLs"].apply(countKeyword)
df.head(10)

,URLs,isPhishing,RawLength,HTTPS,WWW,.COM,largestWord,avgWord,smallestWord,SDlength,countSubDomain,brandName,countBrand,keywordCount
111,13bmosecure.com,1,15,0,0,1,11,7,3,5.656854,1,0,0,2
155,34zy2.sljtm.com,1,15,0,0,1,5,4,3,1.154701,2,0,0,1
108,accountverification.online,1,26,0,0,0,19,12,6,9.192388,1,0,0,3
50,apaypal.co/Secure/PP/Paypal/,1,28,0,0,0,7,5,2,2.408319,1,1,1,1
33,app-findmyiphone.com,1,20,0,0,1,12,6,3,5.196152,1,1,1,3
142,apple-accounts.me/clients/home/,1,31,0,0,0,8,5,2,2.387467,1,1,1,2
39,apple-icloudcid.com,1,19,0,0,1,9,6,3,3.055050,1,1,2,1
54,apple-id-support.info,1,21,0,0,0,7,4,2,2.081666,1,1,1,2
20359,apple-iforget.com,1,17,0,0,1,7,5,3,2.000000,1,1,1,2
166,apple-inc-help.com,1,18,0,0,1,5,4,3,0.957427,1,1,1,4


#### Check for Top Level Domain

In [72]:
def checktld(site):
    result = [0,0,0,0]
    s = re.search("//",site)
    if s:
        site = site[s.end():]
    e = re.search("/",site)
    if e:
        path = site[e.end():]
        site = site[:e.end()-1]
    dot = re.search(".",site)
    site = site[dot.end()-1:]
    arr = re.split(r"\.",site)
    tld = arr[-1]
    www = arr[0]
    if tld in ["com","org","net","gov","in"]:
        return 1
    return 0

df["tld"] = df["URLs"].apply(checktld)


df.head(10)

,URLs,isPhishing,RawLength,HTTPS,WWW,.COM,largestWord,avgWord,smallestWord,SDlength,countSubDomain,brandName,countBrand,keywordCount,tld
111,13bmosecure.com,1,15,0,0,1,11,7,3,5.656854,1,0,0,2,1
155,34zy2.sljtm.com,1,15,0,0,1,5,4,3,1.154701,2,0,0,1,1
108,accountverification.online,1,26,0,0,0,19,12,6,9.192388,1,0,0,3,0
50,apaypal.co/Secure/PP/Paypal/,1,28,0,0,0,7,5,2,2.408319,1,1,1,1,0
33,app-findmyiphone.com,1,20,0,0,1,12,6,3,5.196152,1,1,1,3,1
142,apple-accounts.me/clients/home/,1,31,0,0,0,8,5,2,2.387467,1,1,1,2,0
39,apple-icloudcid.com,1,19,0,0,1,9,6,3,3.055050,1,1,2,1,1
54,apple-id-support.info,1,21,0,0,0,7,4,2,2.081666,1,1,1,2,0
20359,apple-iforget.com,1,17,0,0,1,7,5,3,2.000000,1,1,1,2,1
166,apple-inc-help.com,1,18,0,0,1,5,4,3,0.957427,1,1,1,4,1


#### Check Length of Path

In [73]:
import tldextract

In [74]:
def lenSubdomain(site):
    res = tldextract.extract(site)
    subdomain = res.subdomain
    return (len(subdomain))
df["subdomainLen"] = df["URLs"].apply(lenSubdomain)
df.head(10)

,URLs,isPhishing,RawLength,HTTPS,WWW,.COM,largestWord,avgWord,smallestWord,SDlength,countSubDomain,brandName,countBrand,keywordCount,tld,subdomainLen
111,13bmosecure.com,1,15,0,0,1,11,7,3,5.656854,1,0,0,2,1,0
155,34zy2.sljtm.com,1,15,0,0,1,5,4,3,1.154701,2,0,0,1,1,5
108,accountverification.online,1,26,0,0,0,19,12,6,9.192388,1,0,0,3,0,0
50,apaypal.co/Secure/PP/Paypal/,1,28,0,0,0,7,5,2,2.408319,1,1,1,1,0,0
33,app-findmyiphone.com,1,20,0,0,1,12,6,3,5.196152,1,1,1,3,1,0
142,apple-accounts.me/clients/home/,1,31,0,0,0,8,5,2,2.387467,1,1,1,2,0,0
39,apple-icloudcid.com,1,19,0,0,1,9,6,3,3.055050,1,1,2,1,1,0
54,apple-id-support.info,1,21,0,0,0,7,4,2,2.081666,1,1,1,2,0,0
20359,apple-iforget.com,1,17,0,0,1,7,5,3,2.000000,1,1,1,2,1,0
166,apple-inc-help.com,1,18,0,0,1,5,4,3,0.957427,1,1,1,4,1,0


In [75]:
def lendomain(site):
    res = tldextract.extract(site)
    domain = res.domain
    return (len(domain))
df["domainLen"] = df["URLs"].apply(lendomain)
df.head(10)

,URLs,isPhishing,RawLength,HTTPS,WWW,.COM,largestWord,avgWord,smallestWord,SDlength,countSubDomain,brandName,countBrand,keywordCount,tld,subdomainLen,domainLen
111,13bmosecure.com,1,15,0,0,1,11,7,3,5.656854,1,0,0,2,1,0,11
155,34zy2.sljtm.com,1,15,0,0,1,5,4,3,1.154701,2,0,0,1,1,5,5
108,accountverification.online,1,26,0,0,0,19,12,6,9.192388,1,0,0,3,0,0,19
50,apaypal.co/Secure/PP/Paypal/,1,28,0,0,0,7,5,2,2.408319,1,1,1,1,0,0,7
33,app-findmyiphone.com,1,20,0,0,1,12,6,3,5.196152,1,1,1,3,1,0,16
142,apple-accounts.me/clients/home/,1,31,0,0,0,8,5,2,2.387467,1,1,1,2,0,0,14
39,apple-icloudcid.com,1,19,0,0,1,9,6,3,3.055050,1,1,2,1,1,0,15
54,apple-id-support.info,1,21,0,0,0,7,4,2,2.081666,1,1,1,2,0,0,16
20359,apple-iforget.com,1,17,0,0,1,7,5,3,2.000000,1,1,1,2,1,0,13
166,apple-inc-help.com,1,18,0,0,1,5,4,3,0.957427,1,1,1,4,1,0,14


#### Read Alexa top 1000 sites


In [81]:
alexa = open("Alexa_top_1m.txt", "r")
alexatop = alexa.read()
alexatop1m = alexatop.split('\n')

In [83]:
def isinAlexa(s):
    if s in alexatop1m:
        print (1)
        return 1
    return 0
df["Alexa_top_1m"] = df["URLs"].apply(isinAlexa)
df.head(10)

,URLs,isPhishing,RawLength,HTTPS,WWW,.COM,largestWord,avgWord,smallestWord,SDlength,countSubDomain,brandName,countBrand,keywordCount,tld,subdomainLen,domainLen,Alexa_top_1m
111,13bmosecure.com,1,15,0,0,1,11,7,3,5.656854,1,0,0,2,1,0,11,0
155,34zy2.sljtm.com,1,15,0,0,1,5,4,3,1.154701,2,0,0,1,1,5,5,0
108,accountverification.online,1,26,0,0,0,19,12,6,9.192388,1,0,0,3,0,0,19,0
50,apaypal.co/Secure/PP/Paypal/,1,28,0,0,0,7,5,2,2.408319,1,1,1,1,0,0,7,0
33,app-findmyiphone.com,1,20,0,0,1,12,6,3,5.196152,1,1,1,3,1,0,16,0
142,apple-accounts.me/clients/home/,1,31,0,0,0,8,5,2,2.387467,1,1,1,2,0,0,14,0
39,apple-icloudcid.com,1,19,0,0,1,9,6,3,3.055050,1,1,2,1,1,0,15,0
54,apple-id-support.info,1,21,0,0,0,7,4,2,2.081666,1,1,1,2,0,0,16,0
20359,apple-iforget.com,1,17,0,0,1,7,5,3,2.000000,1,1,1,2,1,0,13,0
166,apple-inc-help.com,1,18,0,0,1,5,4,3,0.957427,1,1,1,4,1,0,14,0


#### Detect gibberish words code

In [86]:
def dash(s):
    if '-' in s:
        return 1
    return 0

def dot(s):
    if '.' in s:
        return 1
    return 0

def slash(s):
    if '/' in s:
        return 1
    return 0

def at(s):
    if '@' in s:
        return 1
    return 0

def question(s):
    if '?' in s:
        return 1
    return 0

def ampersand(s):
    if '&' in s:
        return 1
    return 0

def equal(s):
    if '=' in s:
        return 1
    return 0

def underscore(s):
    if '=' in s:
        return 1
    return 0

df["underscore"] = df["URLs"].apply(underscore)
df["equal"] = df["URLs"].apply(equal)
df["ampersand"] = df["URLs"].apply(ampersand)
df["question"] = df["URLs"].apply(question)
df["at"] = df["URLs"].apply(at)
df["dash"] = df["URLs"].apply(dash)
df["dot"] = df["URLs"].apply(dot)
df["slash"] = df["URLs"].apply(slash)
df.head(10)

,URLs,isPhishing,RawLength,HTTPS,WWW,.COM,largestWord,avgWord,smallestWord,SDlength,...,domainLen,Alexa_top_1m,underscore,equal,ampersand,question,at,dash,dot,slash
111,13bmosecure.com,1,15,0,0,1,11,7,3,5.656854,...,11,0,0,0,0,0,0,0,1,0
155,34zy2.sljtm.com,1,15,0,0,1,5,4,3,1.154701,...,5,0,0,0,0,0,0,0,1,0
108,accountverification.online,1,26,0,0,0,19,12,6,9.192388,...,19,0,0,0,0,0,0,0,1,0
50,apaypal.co/Secure/PP/Paypal/,1,28,0,0,0,7,5,2,2.408319,...,7,0,0,0,0,0,0,0,1,1
33,app-findmyiphone.com,1,20,0,0,1,12,6,3,5.196152,...,16,0,0,0,0,0,0,1,1,0
142,apple-accounts.me/clients/home/,1,31,0,0,0,8,5,2,2.387467,...,14,0,0,0,0,0,0,1,1,1
39,apple-icloudcid.com,1,19,0,0,1,9,6,3,3.055050,...,15,0,0,0,0,0,0,1,1,0
54,apple-id-support.info,1,21,0,0,0,7,4,2,2.081666,...,16,0,0,0,0,0,0,1,1,0
20359,apple-iforget.com,1,17,0,0,1,7,5,3,2.000000,...,13,0,0,0,0,0,0,1,1,0
166,apple-inc-help.com,1,18,0,0,1,5,4,3,0.957427,...,14,0,0,0,0,0,0,1,1,0


In [90]:
def DigDomain(site):
    res = tldextract.extract(site)
    domain = res.domain
    count = 0
    for i in domain:
        if i.isdigit():
            count+=1
    return count

def DigSubDomain(site):
    res = tldextract.extract(site)
    subdomain = res.subdomain
    count = 0
    for i in subdomain:
        if i.isdigit():
            count+=1
    return count

df["countDigitSub"] = df["URLs"].apply(DigSubDomain)
df.head(10)

,URLs,isPhishing,RawLength,HTTPS,WWW,.COM,largestWord,avgWord,smallestWord,SDlength,...,underscore,equal,ampersand,question,at,dash,dot,slash,countDigitDom,countDigitSub
111,13bmosecure.com,1,15,0,0,1,11,7,3,5.656854,...,0,0,0,0,0,0,1,0,2,0
155,34zy2.sljtm.com,1,15,0,0,1,5,4,3,1.154701,...,0,0,0,0,0,0,1,0,0,3
108,accountverification.online,1,26,0,0,0,19,12,6,9.192388,...,0,0,0,0,0,0,1,0,0,0
50,apaypal.co/Secure/PP/Paypal/,1,28,0,0,0,7,5,2,2.408319,...,0,0,0,0,0,0,1,1,0,0
33,app-findmyiphone.com,1,20,0,0,1,12,6,3,5.196152,...,0,0,0,0,0,1,1,0,0,0
142,apple-accounts.me/clients/home/,1,31,0,0,0,8,5,2,2.387467,...,0,0,0,0,0,1,1,1,0,0
39,apple-icloudcid.com,1,19,0,0,1,9,6,3,3.055050,...,0,0,0,0,0,1,1,0,0,0
54,apple-id-support.info,1,21,0,0,0,7,4,2,2.081666,...,0,0,0,0,0,1,1,0,0,0
20359,apple-iforget.com,1,17,0,0,1,7,5,3,2.000000,...,0,0,0,0,0,1,1,0,0,0
166,apple-inc-help.com,1,18,0,0,1,5,4,3,0.957427,...,0,0,0,0,0,1,1,0,0,0


In [91]:
df.to_csv('cleaned_data.csv')





